In [1]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.mime.base import MIMEBase

import smtplib, ssl

import csv

In [2]:
path = "2013\\"
#path = "2014\\"

csv_filename = path+'2013Interacao.csv'
#csv_filename = path+'2014EPCG.csv'

In [3]:
# email server (sending) settings

smtp_server_address = "mail.gpcg.pt"
smtp_server_port = 465  # For SSL

# user credentials
username="publicacoes@gpcg.pt"
password="GPCG2020egdl"


In [4]:
# The message sender
sender_email = "publicacoes@gpcg.pt"

# The message destination cc address
cc_email = "publicacoes@gpcg.pt"


In [5]:
#  The message
message1_subject = "GPCG - {acronym}, paper {articleID}: authorization to publish paper in EG Digital Library"
message1_template = """\
Caros autores do artigo / Dear authors of paper (see english text below),

{title}

publicado no

{event}

o Grupo Português de Computação Gráfica (GPCG) pretende disponibilizar os artigos publicados 
neste evento na Biblioteca Digital da Eurographics.

Para esse efeito necessitamos da autorização dos autores, conforme formulário em anexo. 
Bastará um dos autores autorizar em representação de todos.
Solicitamos que esse autor assine este formulário e envie o respectivo PDF para publicacoes@gpcg.pt

Agradecemos que essa resposta por correio electrónico seja feita respondendo a esta mensagem, 
sem alterar o campo "Assunto" nem o nome do ficheiro contendo o formulário, facilitando assim
o rastreamento automático das respostas que formos recebendo.

Relembramos que a disponibilização destas publicações na biblioteca digital do GPCG
permitirá preservar o acervo bibliográfico do GPCG
e contribuirá para o aumento da visibilidade das mesmas.

Com os melhores cumprimentos,

a Direcção do GPCG

--------------- ENGLISH VERSION -------------------

Dear authors of paper,

{title}

published on

{event}

the Eurographics Portuguese Chapter (Grupo Português de Computação Gráfica - GPCG) 
aims to make available in the Eurographics Digital Library all the papers
published in this event.

We need the authors authorization, as described in the attached form.
It is enough that one of the authors authorizes. 
We request that author to sign this form and send the respective PDF by email to publicacoes@gpcg.pt

Please send the signed form by replying to this message, without changing neither the "Subject" 
nor the form's filename. 
That will help on automatically keeping track of the large number of replies we are expecting.

Making these publications available in the Eurographics Digital Library will preserve 
GPCG's bibliographic collection and will contribute to increase each publication visibility.

Best regards,

GPCG Direction Board"""

In [6]:
#  The message
message_subject = "REMINDER: GPCG - {acronym}, paper {articleID}: authorization to publish paper in EG Digital Library"
message_template = """\
Caros autores do artigo / Dear authors of paper (see english text below),

{title}

publicado no

{event}

vimos por este meio solicitar a vossa atenção para este pedido do GPCG. 
Receber esta autorização é fundamental para que estas actas possam ser publicadas.
Apenas após a recepção de todas as autorizações é que poderemos enviar à Eurographics
a totalidade das actas de {event}

Faltam-nos apenas algumas autorizações,
pelo que agradecemos uma resposta com a brevidade possivel. 

Cumprimentos,

a direcção do GPCG
-----------------------------------------------

o Grupo Português de Computação Gráfica (GPCG) pretende disponibilizar os artigos publicados 
neste evento na Biblioteca Digital da Eurographics.

Para esse efeito necessitamos da autorização dos autores, conforme formulário em anexo. 
Bastará um dos autores autorizar em representação de todos.
Solicitamos que esse autor assine este formulário e envie o respectivo PDF para publicacoes@gpcg.pt

Agradecemos que essa resposta por correio electrónico seja feita respondendo a esta mensagem, 
sem alterar o campo "Assunto" nem o nome do ficheiro contendo o formulário, facilitando assim
o rastreamento automático das respostas que formos recebendo.

Relembramos que a disponibilização destas publicações na biblioteca digital do GPCG
permitirá preservar o acervo bibliográfico do GPCG
e contribuirá para o aumento da visibilidade das mesmas.

Com os melhores cumprimentos,

a Direcção do GPCG

--------------- ENGLISH VERSION -------------------

Dear authors of paper,

{title}

published on

{event}

the Eurographics Portuguese Chapter (Grupo Português de Computação Gráfica - GPCG) 
aims to make available in the Eurographics Digital Library all the papers
published in this event.

We need the authors authorization, as described in the attached form.
It is enough that one of the authors authorizes. 
We request that author to sign this form and send the respective PDF by email to publicacoes@gpcg.pt

Please send the signed form by replying to this message, without changing neither the "Subject" 
nor the form's filename. 
That will help on automatically keeping track of the large number of replies we are expecting.

Making these publications available in the Eurographics Digital Library will preserve 
GPCG's bibliographic collection and will contribute to increase each publication visibility.

Best regards,

GPCG Direction Board"""

In [7]:
# Read the CSV header and identify relevant columns (this is row 0)

with open(csv_filename,"rt") as csv_in:

    csv_col = {}  # our dictionary of csv columns
    
    for rownbr, row in enumerate (csv.reader(csv_in, delimiter=";")):
        
        csv_col['ArticleID'] = row.index ('ArticleID')
        csv_col['Type'] = row.index ('Type')
        csv_col['Title'] = row.index ('Title')
        csv_col['State'] = row.index ('State')
        csv_col['Acronym'] = row.index ('Acronym')
        csv_col['Event'] = row.index ('Event')
        csv_col['File'] = row.index ('File')
        csv_col['LicenseFile'] = row.index ('LicenseFile')
        csv_col['AuthorsContact'] = []
        for nbr_au in range(10):
            field_name = 'Author'+str(nbr_au)+'Contact'
            if field_name in row:
                csv_col['AuthorsContact'].append(row.index (field_name))
        
        break   # process only line 0
    
print(csv_col)


{'ArticleID': 0, 'Type': 1, 'Title': 2, 'State': 5, 'Acronym': 18, 'Event': 16, 'File': 14, 'LicenseFile': 15, 'AuthorsContact': [8, 21, 25, 29, 33, 37, 41, 45, 49]}


In [8]:
# Create a secure SSL context
context = ssl.create_default_context()

# login
try:
    server = smtplib.SMTP_SSL(smtp_server_address, smtp_server_port, context=context)
    server.login(username, password)
except:
    print ("Unexpected error creating and logging to SMTP server. Terminate!")
    raise  

In [9]:

with open(csv_filename,"rt") as csv_in:
    
    for rownbr, row in enumerate (csv.reader(csv_in, delimiter=";")):
        
        if rownbr==0: continue    # skip the header

        if row[csv_col['Type']] != 'Article': continue # process only Articles
        if row[csv_col['State']] != 'email enviado': continue # process only Articles
        if int(row[csv_col['ArticleID']]) <= 28: continue # process only Articles
            
        # Build the MIME message
        message = MIMEMultipart()
        message["Subject"] = message_subject.format(acronym=row[csv_col["Acronym"]],
                                                   articleID=row[csv_col["ArticleID"]])
        message["From"] = sender_email
        message["Cc"] = cc_email

        # collect all authors emails
        destination_email = []
        to_str = ""
        for au_col in csv_col['AuthorsContact']:
            au_email = row[au_col]
            if au_email == "": continue
                
            if len(destination_email) > 0:   # not the first email address
                to_str = to_str + "; "
            to_str = to_str + au_email
            destination_email.append (au_email)
        
        if len(destination_email)==0:  # empty list of destinations
            continue
        message["To"] = to_str    

        message_text = message_template.format(title=row[csv_col['Title']],
                                              event=row[csv_col['Event']],)
        message.attach(MIMEText(message_text,"plain"))
    
        # read the attached files
        for filename in [row[csv_col['File']],row[csv_col['LicenseFile']]]:
            # Open PDF file in binary mode
            fN = path + filename
            with open(fN, "rb") as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())
        
            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)

            # Add header as key/value pair to attachment part
            part.add_header(
                "Content-Disposition",
                f"attachment; filename= {filename}",
            )
    
            # Add attachment to message 
            message.attach(part)
        # attachment PDFs DONE
    
        #convert message to string
        str_msg = message.as_string()
    
        # send message
        destination_email.append(cc_email)
        server.sendmail(sender_email, destination_email, str_msg)
        print ('Sent email for paper {0}'.format(row[csv_col["ArticleID"]]))
        
        # debug
        #print (destination_addresses)
        #print(message["Subject"])
        #print(message_text)
    # all rows of CSV done
# CSV closed


Sent email for paper 31
Sent email for paper 32
Sent email for paper 33
Sent email for paper 35
Sent email for paper 36
Sent email for paper 37
Sent email for paper 38
Sent email for paper 39
Sent email for paper 42


In [10]:
# close the SMTP connection
server.quit()

(221, b'cp29.webserver.pt closing connection')

In [11]:
print ("That's all, folks!")

That's all, folks!
